In [1]:
from skmob.utils.plot import plot_gdf
from skmob.tessellation import tilers
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from skmob.utils import utils
from math import pi, asin, sin, cos, sqrt

pd.set_option('display.max_columns', 500)



%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.offsetbox import AnchoredText
plt.rcParams["figure.figsize"] = (12, 9) # (w, h)

In [2]:
def distance_earth_km(src, dest):

    lat1, lat2 = src['lat']*pi/180, dest['lat']*pi/180
    lon1, lon2 = src['lon']*pi/180, dest['lon']*pi/180
    dlat, dlon = lat1-lat2, lon1-lon2

    ds = 2 * asin(sqrt(sin(dlat/2.0) ** 2 + cos(lat1) * cos(lat2) * sin(dlon/2.0) ** 2))
    return 6371.01 * ds

In [3]:
def compute_distance_matrix(tess):

    lats_lngs = tess.geometry.apply(utils.get_geom_centroid, args=[True]).values
    
    distance_matrix = np.zeros((len(tess),len(tess)))

    for i in range(0,len(tess)):
        for j in range(0,len(tess)):
            if i != j:
                d = distance_earth_km({'lat':lats_lngs[i][0],'lon':lats_lngs[i][1]},
                                         {'lat':lats_lngs[j][0],'lon':lats_lngs[j][1]})
                distance_matrix[i,j] = d
                
    return distance_matrix

In [4]:
tess_nyc = gpd.read_file('NYC_tessellation.geojson')

In [5]:
plot_gdf(tess_nyc, style_func_args={'fillColor':'gray', 'color':'black', 'opacity': 0.2}, zoom = 9) 

In [6]:
dist = compute_distance_matrix(tess_nyc)
dist

array([[ 0.        ,  1.38367382,  3.09427763, ..., 16.64918734,
        22.99880201, 24.17048728],
       [ 1.38367382,  0.        ,  1.95699301, ..., 15.64278349,
        22.28188943, 23.48949201],
       [ 3.09427763,  1.95699301,  0.        , ..., 15.76599289,
        22.87606127, 24.13321915],
       ...,
       [16.64918734, 15.64278349, 15.76599289, ...,  0.        ,
         8.28498019,  9.66490499],
       [22.99880201, 22.28188943, 22.87606127, ...,  8.28498019,
         0.        ,  1.3799248 ],
       [24.17048728, 23.48949201, 24.13321915, ...,  9.66490499,
         1.3799248 ,  0.        ]])

In [7]:
np.save("dist_mat_NYC.npy", dist)